In [1]:
from ROOT import gSystem, TChain, TLorentzVector, TFile, TTree, TRandom2, gInterpreter
import time
import numpy as np


Welcome to JupyROOT 6.19/01


In [2]:
gInterpreter.AddIncludePath("/home/lihuang/tools/MG5_aMC_v2_6_6/Delphes/")
gInterpreter.AddIncludePath("/home/lihuang/tools/MG5_aMC_v2_6_6/Delphes/external/")
gInterpreter.Declare('#include "/home/lihuang/tools/MG5_aMC_v2_6_6/Delphes/classes/DelphesClasses.h"')

gInterpreter.Declare('#include "/home/lihuang/tools/MG5_aMC_v2_6_6/Delphes/external/ExRootAnalysis/ExRootAnalysisLinkDef.h"')

gSystem.Load("/home/lihuang/tools/MG5_aMC_v2_6_6/Delphes/libDelphes.so")


gInterpreter.Declare('#include "/home/lihuang/tools/MG5_aMC_v2_6_6/ExRootAnalysis/ExRootAnalysis/ExRootClasses.h"')
gSystem.Load('/home/lihuang/tools/root6/lib/libHist.so')
gSystem.Load('/home/lihuang/tools/root6/lib/libGraf.so')
gSystem.Load('/home/lihuang/tools/root6/lib/libGpad.so')

gSystem.Load('/home/lihuang/tools/MG5_aMC_v2_6_6/ExRootAnalysis/libExRootAnalysis.so')


0

Warning in <TClassTable::Add>: class ExRootTreeReader already in TClassTable
Warning in <TClassTable::Add>: class ExRootTreeWriter already in TClassTable
Warning in <TClassTable::Add>: class ExRootTreeBranch already in TClassTable
Warning in <TClassTable::Add>: class ExRootResult already in TClassTable
Warning in <TClassTable::Add>: class ExRootClassifier already in TClassTable
Warning in <TClassTable::Add>: class ExRootFilter already in TClassTable


In [3]:
def smear(particle, PID):
    if(PID=="quark"):
        E, px, py, pz = particle.E(), particle.Px(), particle.Py(), particle.Pz()
        sigma = ((5.3/E)**2+(0.74/E**0.5)**2+0.05)**0.5
        new_E = np.random.normal(E, E*sigma)
        ratio = new_E/E
        new_px, new_py, new_pz = px*ratio, py*ratio, pz*ratio
        new_particle = TLorentzVector()
        new_particle.SetPxPyPzE(new_px, new_py, new_pz, new_E)
        if(new_E<=0):
            new_particle.SetPxPyPzE(0, 0, 0, 0)
        
    if(PID=="electron"):
        E, px, py, pz = particle.E(), particle.Px(), particle.Py(), particle.Pz()
        sigma = 0.3/E+0.1/E**0.5+0.01
        new_E = np.random.normal(E, E*sigma)
        ratio = new_E/E
        new_px, new_py, new_pz = px*ratio, py*ratio, pz*ratio
        new_particle = TLorentzVector()
        new_particle.SetPxPyPzE(new_px, new_py, new_pz, new_E)
        if(new_E<=0):
            new_particle.SetPxPyPzE(0, 0, 0, 0)

    if(PID=="muon"):
        E, px, py, pz = particle.E(), particle.Px(), particle.Py(), particle.Pz()
        sigma_ID = (0.023035**2+(0.000347*E)**2)**0.5
        sigma_MS = ((0.12/E)**0.5+0.03278**2+(0.00014*E)**2)**0.5
        sigma = sigma_ID*sigma_MS/(sigma_ID**2+sigma_MS**2)**0.5
        new_E = np.random.normal(E, E*sigma)
        ratio = new_E/E
        new_px, new_py, new_pz = px*ratio, py*ratio, pz*ratio
        new_particle = TLorentzVector()
        new_particle.SetPxPyPzE(new_px, new_py, new_pz, new_E)
        if(new_E<=0):
            new_particle.SetPxPyPzE(0, 0, 0, 0)

    return new_particle, sigma

In [4]:
chain1 = TChain("LHEF")
chain1.Add('/mnt/10Ti/lihuang/combinatory/data/mg5_data/pp2tt/Events/run_01_0/unweighted_events.root')
print ("The event number is", chain1.GetEntries())


The event number is 1000000


In [5]:
%%time
start_time = time.time()
result1 = []

for count, event in enumerate(chain1):
    if(count==10000): break
    b1, b2, l1, l2 = TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector()
    neu1, neu2 = TLorentzVector(), TLorentzVector()
    for particle in event.Particle:        
        if(particle.PID==5 and particle.Status==1):
            b1.SetPxPyPzE(particle.Px, particle.Py, particle.Pz, particle.E)
        if(particle.PID==-5 and particle.Status==1):
            b2.SetPxPyPzE(particle.Px, particle.Py, particle.Pz, particle.E)
        if((particle.PID==-11 or particle.PID==-13) and particle.Status==1):
            l1.SetPxPyPzE(particle.Px, particle.Py, particle.Pz, particle.E)
        if((particle.PID==11 or particle.PID==13) and particle.Status==1):
            l2.SetPxPyPzE(particle.Px, particle.Py, particle.Pz, particle.E)
        if((particle.PID==12 or particle.PID==14) and particle.Status==1):
            neu1.SetPxPyPzE(particle.Px, particle.Py, particle.Pz, particle.E)
        if((particle.PID==-12 or particle.PID==-14) and particle.Status==1):
            neu2.SetPxPyPzE(particle.Px, particle.Py, particle.Pz, particle.E)
        met = neu1+neu2

    #print ((b1+b2+l1+l2).Pt(), met.Pt())
    result1.append([b1.E(), b1.Px(), b1.Py(), b1.Pz(), 
                   l1.E(), l1.Px(), l1.Py(), l1.Pz(), 
                   b2.E(), b2.Px(), b2.Py(), b2.Pz(),
                   l2.E(), l2.Px(), l2.Py(), l2.Pz(),
                   (b1+l1).M(), b1.DeltaR(l1),
                   (b2+l2).M(), b2.DeltaR(l2),
                   (b2+l1).M(), b2.DeltaR(l1),
                   (b1+l2).M(), b1.DeltaR(l2),
                   met.Px(), met.Py()])

print("Time: %.2f" %((time.time()-start_time)/3600.))

result1 = np.array(result1)

np.save("./pp_tt_parton14TeV", result1)

Time: 0.00
CPU times: user 638 ms, sys: 149 ms, total: 787 ms
Wall time: 782 ms


In [6]:
%%time
start_time = time.time()
result = []

sigma_ = []
for count, event in enumerate(chain1):
    if(count==10000): break
    b1, b2, l1, l2 = TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector()
    neu1, neu2 = TLorentzVector(), TLorentzVector()
    for particle in event.Particle:        
        if(particle.PID==5 and particle.Status==1):
            b1.SetPxPyPzE(particle.Px, particle.Py, particle.Pz, particle.E)
            b1, _ = smear(b1, PID="quark")
        if(particle.PID==-5 and particle.Status==1):
            b2.SetPxPyPzE(particle.Px, particle.Py, particle.Pz, particle.E)
            b2, _ = smear(b2, PID="quark")
        if((particle.PID==-11 or particle.PID==-13) and particle.Status==1):
            l1.SetPxPyPzE(particle.Px, particle.Py, particle.Pz, particle.E)
            if(particle.PID==-11):
                l1, _ = smear(l1, PID="electron")
            if(particle.PID==-13):
                l1, _ = smear(l1, PID="muon")
        if((particle.PID==11 or particle.PID==13) and particle.Status==1):
            l2.SetPxPyPzE(particle.Px, particle.Py, particle.Pz, particle.E)
            if(particle.PID==11):
                l2, _ = smear(l2, PID="electron")
            if(particle.PID==13):
                l2, _ = smear(l2, PID="muon")

    tot = b1+b2+l1+l2

    #print ((b1+b2+l1+l2).Pt(), met.Pt())
    result.append([b1.E(), b1.Px(), b1.Py(), b1.Pz(), 
                   l1.E(), l1.Px(), l1.Py(), l1.Pz(), 
                   b2.E(), b2.Px(), b2.Py(), b2.Pz(),
                   l2.E(), l2.Px(), l2.Py(), l2.Pz(),
                   (b1+l1).M(), b1.DeltaR(l1),
                   (b2+l2).M(), b2.DeltaR(l2),
                   (b2+l1).M(), b2.DeltaR(l1),
                   (b1+l2).M(), b1.DeltaR(l2),
                   -tot.Px(), -tot.Py()])

print("Time: %.2f" %((time.time()-start_time)/3600.))

result = np.array(result)

np.save("./pp_tt_parton14TeV_smearing", result)


Time: 0.00
CPU times: user 881 ms, sys: 76 ms, total: 957 ms
Wall time: 950 ms
